In [0]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import Callback
from keras.layers.merge import concatenate, add
from keras.layers import Dense, Embedding, CuDNNLSTM, SpatialDropout1D,Conv2D, Conv1D, Flatten,MaxPooling1D, Input, Bidirectional, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler as SS
import re
from sklearn.preprocessing import OneHotEncoder as OE
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from sklearn.metrics import f1_score

In [3]:
from google.colab import files
uploaded = files.upload()

Saving train_2kmZucJ.csv to train_2kmZucJ (1).csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['train_2kmZucJ.csv']))

In [0]:
df = df.fillna(" ")

In [6]:
df.head()

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,0,Finally a transparant silicon case ^^ Thanks t...
2,0,We love this! Would you go? #talk #makememorie...
3,0,I'm wired I know I'm George I was made that wa...
4,1,What amazing service! Apple won't even talk to...


In [0]:
df['tweet'] = df['tweet'].apply(lambda x: x.lower())
df['tweet'] = df['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
df['tweet'] = df['tweet'].apply((lambda x: re.sub(r"http\S+", "", x)))

In [12]:
X.shape

(7920, 48)

1999

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['tweet'].values)
X = tokenizer.texts_to_sequences(df['tweet'].values)
X = pad_sequences(X)

In [0]:
Y = np.array(df['label'])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

In [0]:
#now that we have our train and test values let us define our LSTM model : 

In [63]:
embed_dim = 100
lstm_out = 100
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(CuDNNLSTM(lstm_out, return_sequences=True))
model.add(CuDNNLSTM(lstm_out, return_sequences=True))
model.add(CuDNNLSTM(lstm_out, return_sequences=True))
model.add(CuDNNLSTM(lstm_out, return_sequences=True))
model.add(CuDNNLSTM(lstm_out))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='rmsprop',metrics = ['accuracy'])
'''
def ConvNet(max_sequence_length, num_words, embedding_dim, trainable=False, extra_conv=True):
    #the filter
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            input_length=max_sequence_length,
                            )

    #the unknown image
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    #the merge function of the first convolution 
    embedded_sequences = embedding_layer(sequence_input)
    

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [10,20,30] # in the loop, first apply 3 as size, then 4 then 5

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        #kernel is the filter
        l_pool = MaxPooling1D(pool_size=4)(l_conv)
        convs.append(l_pool)

    l_merge = concatenate(convs, axis=1)

    
    # activated if extra_convoluted is true at the def
    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=4, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=4)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(1000, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    # Finally, we feed the output into a Sigmoid layer.
    # The reason why sigmoid is used is because we are trying to achieve a binary classification(1,0) 
    # for each of the 6 labels, and the sigmoid function will squash the output between the bounds of 0 and 1.
    preds = Dense(1, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
    return model
model = ConvNet(X.shape[1], max_fatures,embed_dim)'''

"\ndef ConvNet(max_sequence_length, num_words, embedding_dim, trainable=False, extra_conv=True):\n    #the filter\n    embedding_layer = Embedding(num_words,\n                            embedding_dim,\n                            input_length=max_sequence_length,\n                            )\n\n    #the unknown image\n    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')\n    #the merge function of the first convolution \n    embedded_sequences = embedding_layer(sequence_input)\n    \n\n    # Yoon Kim model (https://arxiv.org/abs/1408.5882)\n    convs = []\n    filter_sizes = [10,20,30] # in the loop, first apply 3 as size, then 4 then 5\n\n    for filter_size in filter_sizes:\n        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)\n        #kernel is the filter\n        l_pool = MaxPooling1D(pool_size=4)(l_conv)\n        convs.append(l_pool)\n\n    l_merge = concatenate(convs, axis=1)\n\n    \n    # activated if extra_

In [0]:
class Metrics(Callback):
  def on_train_begin(self, logs={}):
    self.val_f1s = []
    
    
  def on_epoch_end(self, epoch, logs={}):
    val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
    val_targ = self.model.validation_data[1]
    _val_f1 = f1_score(val_targ, val_predict)
    self.val_f1s.append(_val_f1)
    print ("— val_f1: %f"%(_val_f1))
    return

  
  
metrics = Metrics()

In [65]:
batch_size = 32
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 30,  verbose = 2, validation_data=(X_test,Y_test),callbacks=[mcp_save, earlyStopping])

Train on 5306 samples, validate on 2614 samples
Epoch 1/30
 - 8s - loss: 0.4025 - acc: 0.8174 - val_loss: 0.2817 - val_acc: 0.8761
Epoch 2/30
 - 3s - loss: 0.2536 - acc: 0.8909 - val_loss: 0.2665 - val_acc: 0.8806
Epoch 3/30
 - 3s - loss: 0.2283 - acc: 0.9046 - val_loss: 0.2584 - val_acc: 0.8871
Epoch 4/30
 - 3s - loss: 0.2134 - acc: 0.9084 - val_loss: 0.2651 - val_acc: 0.8803
Epoch 5/30
 - 3s - loss: 0.2045 - acc: 0.9159 - val_loss: 0.2475 - val_acc: 0.8944
Epoch 6/30
 - 3s - loss: 0.1982 - acc: 0.9171 - val_loss: 0.2476 - val_acc: 0.8959
Epoch 7/30
 - 3s - loss: 0.1909 - acc: 0.9212 - val_loss: 0.2589 - val_acc: 0.8933
Epoch 8/30
 - 3s - loss: 0.1898 - acc: 0.9237 - val_loss: 0.2621 - val_acc: 0.8887
Epoch 9/30
 - 3s - loss: 0.1792 - acc: 0.9297 - val_loss: 0.2597 - val_acc: 0.8910
Epoch 10/30
 - 3s - loss: 0.1707 - acc: 0.9322 - val_loss: 0.3608 - val_acc: 0.8722
Epoch 11/30
 - 3s - loss: 0.1644 - acc: 0.9367 - val_loss: 0.2555 - val_acc: 0.8879
Epoch 12/30
 - 3s - loss: 0.1537 - ac

In [0]:
model.load_weights(filepath = '.mdl_wts.hdf5')

In [18]:
from google.colab import files
uploaded = files.upload()

Saving test_oJQbWVk.csv to test_oJQbWVk (2).csv


In [67]:
model.evaluate(X_test, Y_test)

2614/2614 [==============================] - 1s 313us/step


[0.24754307896195246, 0.8944146901300689]

In [0]:
import io
df_test= pd.read_csv(io.BytesIO(uploaded['test_oJQbWVk.csv']))
df_test = df_test.fillna(" ")
#tokenizer.fit_on_texts(df_test['tweet'].values)
df_test['tweet'] = df_test['tweet'].apply(lambda x: x.lower())
df_test['tweet'] = df_test['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
df_test['tweet'] = df_test['tweet'].apply((lambda x: re.sub(r"http\S+", "", x)))
X_TEST = tokenizer.texts_to_sequences(df_test['tweet'].values)
X_TEST = pad_sequences(X_TEST, maxlen=48)
Y_PRED = model.predict(X_TEST)
y_pred = []
for i in Y_PRED:
  y_pred.append(int(np.round(i)))

In [0]:
result_df = pd.DataFrame(y_pred, columns =['label'])
result_df.to_csv("result_combinedCNNLSTMimprovedmodel2.csv")
from google.colab import files
files.download("result_combinedCNNLSTMimprovedmodel2.csv")